In [9]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-dilavar7517')

In [13]:
latitude = 55.7558
longitude = 37.6176
region = ee.Geometry.Point([longitude, latitude], proj=ee.Projection('EPSG:4326'))

In [15]:
start_date = '2024-09-01'
end_date = '2024-09-31'

era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date) \
    .select(['mean_2m_air_temperature', 'total_precipitation', 'mean_2m_dewpoint_temperature'])

era5_mean = era5.mean()
data = era5_mean = era5_mean.reproject(crs='EPSG:4326', scale=1000)

temperature = data['mean_2m_air_temperature'] - 273.15
dewpoint = data['mean_2m_dewpoint_temperature'] - 273.15

import math

def calculate_relative_humidity(t_air, t_dew):
    rh = 100 * (math.exp((17.625 * t_dew)/(243.04 + t_dew)) / math.exp((17.625 * t_air)/(243.04 + t_air)))
    return rh

humidity = calculate_relative_humidity(temperature, dewpoint)

print(f"Средняя температура: {temperature:.2f} °C")
print(f"Относительная влажность: {humidity:.2f} %")

TypeError: 'Image' object is not subscriptable

In [6]:
modis = ee.ImageCollection('MODIS/006/MOD11A1') \
    .filterDate(start_date, end_date) \
    .select('LST_Day_1km')

modis_mean = modis.mean()
lst = modis_mean.sample(region, scale=1000).first().get('LST_Day_1km').getInfo()
surface_temp = lst * 0.02 - 273.15
print(f"Температура поверхности: {surface_temp:.2f} °C")

EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [7]:
soil = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

soil_type = soil.sample(region, scale=250).first().get('b0').getInfo()

soil_types = {
    1: 'clay',
    2: 'sandy clay',
    3: 'silty clay',
    4: 'clay loam',
    5: 'sandy clay loam',
    6: 'silty clay loam',
    7: 'loam',
    8: 'sandy loam',
    9: 'silt loam',
    10: 'loamy sand',
    11: 'sand',
    12: 'silt',
    13: 'peat',
    14: 'chalk'
}

soil_type_name = soil_types.get(int(soil_type), 'Unknown')

print(f"Тип почвы: {soil_type_name}")

EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [16]:
wind = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date) \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

wind_mean = wind.mean()

def calculate_wind_speed(u, v):
    return math.sqrt(u**2 + v**2)

wind_data = wind_mean.sample(region, scale=1000).first().getInfo()['properties']
u_wind = wind_data['u_component_of_wind_10m']
v_wind = wind_data['v_component_of_wind_10m']
wind_speed = calculate_wind_speed(u_wind, v_wind)

print(f"Скорость ветра: {wind_speed:.2f} м/с")

EEException: Image.sample: Unable to find a crs.

In [17]:
precipitation = data['total_precipitation'] 
rain = precipitation > 0

print(f"Наличие дождя: {'Да' if rain else 'Нет'}")

TypeError: 'Image' object is not subscriptable

In [ ]:
s5p = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2') \
    .filterDate(start_date, end_date)

s5p_mean = s5p.mean()
no2 = s5p_mean.sample(region, scale=1000).first().get('NO2_column_number_density').getInfo()

print(f"Концентрация NO₂: {no2:.2e} mol/m²")